In [6]:
import json, ast, pandas as pd
from pathlib import Path

In [7]:
def parse_payload_line(line: str):
    """Extrae dict del payload a partir de la línea 'Payload: {...}'."""
    txt = line.split("Payload:", 1)[1].strip()
    try:
        return ast.literal_eval(txt)  # dict estilo Python
    except Exception:
        try:
            return json.loads(
                txt.replace("'", '"')
                   .replace(" True", " true")
                   .replace(" False", " false")
                   .replace(" None", " null")
            )
        except Exception:
            return {}

def parse_response_line(line: str):
    """Extrae dict de la línea 'Response JSON: {...}'."""
    txt = line.split("Response JSON:", 1)[1].strip()
    try:
        return ast.literal_eval(txt)
    except Exception:
        try:
            return json.loads(
                txt.replace("'", '"')
                   .replace(" True", " true")
                   .replace(" False", " false")
                   .replace(" None", " null")
            )
        except Exception:
            return {}

In [8]:
log_path = Path("../logs/logistica/logistica.log")  # ajusta según tu notebook
lines = log_path.read_text(encoding="utf-8").splitlines()

events, current = [], None

for line in lines:
    if "Incoming request: POST /entregas" in line:
        current = {"direccion_original": None, "direccion_guardada": None}
    if not current:
        continue
    if "Payload:" in line:
        current["direccion_original"] = parse_payload_line(line).get("direccion")
    if "Response JSON:" in line:
        current["direccion_guardada"] = parse_response_line(line).get("direccion")
    if 'werkzeug' in line and '"POST /entregas' in line:
        current["test_ok"] = current["direccion_original"] != current["direccion_guardada"]
        events.append(current)
        current = None

In [4]:
# DataFrame con resultados
df = pd.DataFrame(events)
display(df)

# resumen
total = len(events)
passed = sum(1 for e in events if e["test_ok"])
print(f"Total POSTs: {total}, Cifrados OK: {passed}, Cumplimiento: {passed/total*100:.2f}%")

,direccion_original,direccion_guardada,test_ok
0,"Calle 123 #45-67, Bogotá",a8347e43ea1a3e7a4209e3bec51ccdb1:3aee7ff1b410b...,True
1,"Calle 123 #45-67, Bogotá",52596862a69339915d2e2ce2bc0f75ab:4cc2affe91166...,True
2,"Calle 123 #45-67, Bogotá",217cd26fcda2fcd7d1c44d6fa776ad0c:2e1bbf7c656e8...,True
3,"Calle 123 #45-67, Bogotá",ed6cbd807d4365615d09fb7e25b8fc89:26d524fe0355a...,True
4,"Calle 123 #45-67, Bogotá",3c49a6a980d0c8f244953d85b54060b8:0de6b62dd1e4d...,True
...,...,...,...
241,"Calle 123 #45-67, Bogotá",b773bacd091bcf88c08ee86457db64e3:d163cba6d96a1...,True
242,"Calle 123 #45-67, Bogotá",37270924ab9537cd5dd1d5a215ed6d40:e170be4ff005a...,True
243,"Calle 123 #45-67, Bogotá",c50890229dc7ef9ca28cd3bbb154e58a:3e36d902b9f0f...,True
244,"Calle 123 #45-67, Bogotá",f2412ab4ae68e8968c521c0cbee81a73:b182fdcc3586c...,True


Total POSTs: 246, Cifrados OK: 246, Cumplimiento: 100.00%


In [9]:
# Resumen general de POSTs
summary = (
    df.groupby("test_ok")
      .size()
      .reset_index(name="count")
      .rename(columns={"test_ok": "cifrado_ok"})
)

summary["pass_rate_%"] = (summary["count"] / summary["count"].sum() * 100).round(2)

display(summary)

,cifrado_ok,count,pass_rate_%
0,True,246,100.0
